In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
y_train = train_df.target
X_train = train_df.drop(columns=["target"])

In [4]:
import re
def remove_digits(DF):
    for i in np.arange(len(DF)):
        DF[i] = re.sub(r"\d+", "", DF[i], flags=re.UNICODE)

In [5]:
X_train = X_train.title
remove_digits(X_train)

In [6]:
test_df.head(10)

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67
5,135314,ryazan.gorodrabot.ru,"Вакансия Мерчендайзер визитный в Рязани, Групп..."
6,135315,m.fotosklad.ru,Смартфон Apple iPhone XR 64GB Черный A2105 EU ...
7,135316,sim-dealer.ru,Духовой шкаф Siemens HB337GYS0R - Sim-Dealer. ...
8,135317,krasnoyarsk.hh.ru,Вакансия Аналитик Департамента внедрения инфор...
9,135318,vsetop.org,Anomaly Defenders v1.0 – торрент


In [7]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english') + stopwords.words('russian'))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=stop_words)

In [9]:
X_train_vectorized = vectorizer.fit_transform(X_train)
feature_names = np.array(vectorizer.get_feature_names())
for i in np.arange(len(X_train)):
    x_vector = X_train_vectorized.getrow(i).toarray()[0]
    X_train[i] = [feature for feature in feature_names[x_vector > 0]]

In [10]:
def vectorize(DF, vectorizer):
    DF_vectorized = vectorizer.transform(DF)
    feature_names = np.array(vectorizer.get_feature_names())
    for i in np.arange(len(DF)):
        x_vector = DF_vectorized.getrow(i).toarray()[0]
        DF[i] = [feature for feature in feature_names[x_vector > 0]]

In [85]:
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec
size = 100

In [86]:
w2v_model = Word2Vec(X_train, min_count = 1, workers = cpu_count(), size=size) 
w2v_model.train(X_train, total_examples = w2v_model.corpus_count, epochs = 3) 

(2950854, 3082431)

In [87]:
w2v_model.wv.most_similar('boobs')

[('amateur', 0.9977993369102478),
 ('cumshot', 0.9962483644485474),
 ('cum', 0.9956215023994446),
 ('blonde', 0.9955916404724121),
 ('bbw', 0.9953222870826721),
 ('butt', 0.9951209425926208),
 ('dildo', 0.994665265083313),
 ('cowgirl', 0.9944596886634827),
 ('dick', 0.9939305782318115),
 ('asian', 0.9938560724258423)]

In [88]:
def convert(data):
    X = np.zeros((len(data), size))
    for i in np.arange(len(data)):
        for word in data[i]:
            length = len(data[i])
            X[i] += w2v_model.wv[word]/length
    return X
X = convert(X_train)
print(X)

[[ 1.04584338e-01 -2.55662730e-01  8.27407924e-02 ...  2.36622254e-01
  -1.98850208e-01  6.57837010e-01]
 [-6.42691360e-02 -1.95368196e-01  4.70723593e-02 ...  5.95994020e-03
  -3.60606212e-01  3.18254547e-01]
 [ 1.58372413e-01 -2.40983504e+00  2.83045888e+00 ... -8.27999495e-01
  -1.64244153e+00  1.42996362e+00]
 ...
 [ 8.27458318e-01 -1.20626860e+00 -8.42313553e-02 ...  5.39287671e-01
   3.91442514e-01 -6.64962921e-04]
 [ 1.60203813e+00 -3.80611377e-01  1.03514545e+00 ... -1.29794353e-02
  -6.25451255e-01  1.97928884e+00]
 [ 1.83822211e+00 -2.51110767e-01  1.19899915e+00 ... -1.84408695e-01
  -7.10842393e-01  2.69942693e+00]]


In [ ]:
X_test = test_df.drop(columns=[])
X_test = X_test.title
remove_digits(X_test)
vectorize(X_test, vectorizer)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(verbose=1, n_jobs=cpu_count())
forest_model.fit(X, y_train)

In [23]:
from sklearn.metrics import f1_score
y_pred = forest_model.predict(X)
print(f1_score(y_pred, y_train))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished


0.9935611987002047


In [89]:
Y = convert(X_test)

In [25]:
test_df["target"] = forest_model.predict(Y).astype(bool)
test_df[["id", "target"]].to_csv("Smirnov01.csv", index=False)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.6s finished
